In [1]:
!pip install pyarrow

In [1]:
import sklearn

In [2]:
sklearn.__version__

'1.0.2'

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [21]:
import pickle

In [5]:
# Modularizing the codes
def read_dataframe(filename):
    df = pd.read_parquet(filename, engine="pyarrow")
    
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ["PULocationID", "DOLocationID"]
    
    
    df[categorical] = df[categorical].astype(str)
    return df

In [6]:
df_train = read_dataframe("green_tripdata_2021-01.parquet")
df_val = read_dataframe("green_tripdata_2021-02.parquet")

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [16]:
df_train["PU_DO"] = df_train["PULocationID"] + "_"+ df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_"+ df_val["DOLocationID"]

In [18]:
categorical = ["PU_DO"]#,"PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [11]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [22]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.4793657829373155

In [20]:
lr = Lasso(alpha=0.001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

9.233436225720547

In [23]:
# Saving the model
with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)